In [4]:
import torch
from transformers import AutoModelForImageClassification, AutoImageProcessor
from torchvision import transforms
from PIL import Image
import ollama
import os

class FoodImageClassifier:
    CONFIDENCE_THRESHOLD = 0.5

    def __init__(self, model_name='llama3.2:1b'):
        self.model_name = model_name

        # Load the general category model and processor
        self.category_processor = AutoImageProcessor.from_pretrained("Kaludi/food-category-classification-v2.0")
        self.category_model = AutoModelForImageClassification.from_pretrained("Kaludi/food-category-classification-v2.0")

        # Load item-specific models
        self.fruit_veg_model = AutoModelForImageClassification.from_pretrained("jazzmacedo/fruits-and-vegetables-detector-36")
        
        # Map category labels to specific item models
        self.category_to_model = {
            "Fruit": self.fruit_veg_model,
            "Vegetable": self.fruit_veg_model
        }

        # Define preprocessing transformations
        self.preprocess = transforms.Compose([
            transforms.Resize(224),
            transforms.CenterCrop(224),
            transforms.ToTensor()
        ])

    def _validate_image(self, image: Image.Image) -> Image.Image:
        """
        Ensure the image is in JPEG format. If not, convert it to JPEG.
        
        Parameters:
        image (PIL.Image.Image): The input image.

        Returns:
        PIL.Image.Image: A JPEG format image.
        """
        if image.format != 'JPEG':
            image = image.convert("RGB")  # Ensure RGB format for JPEG compatibility
            image.save("temp_image.jpg", "JPEG")
            image = Image.open("temp_image.jpg")  # Reopen as JPEG format
        return image

    def classify_images(self, images: list[Image.Image]) -> list[tuple]:
        """
        Accepts a list of images, preprocesses, and classifies them into categories and specific items.
        
        Parameters:
        images (list[PIL.Image.Image]): List of images.

        Returns:
        list: List of (category, item) tuples for each image.
        """
        validated_images = [self._validate_image(img) for img in images]
        preprocessed_images = [self.preprocess(img).unsqueeze(0) for img in validated_images]
        batch = torch.cat(preprocessed_images, dim=0)

        results = []
        with torch.no_grad():
            category_outputs = self.category_model(batch)

        for idx, category_logits in enumerate(category_outputs.logits):
            category_idx = torch.argmax(category_logits).item()
            category_label = self.category_model.config.id2label[category_idx]

            item_model = self.category_to_model.get(category_label)
            if item_model is None:
                results.append((category_label, None))
                continue

            with torch.no_grad():
                item_logits = item_model(batch[idx].unsqueeze(0))
                item_probs = torch.softmax(item_logits.logits, dim=1)
                item_confidence, item_idx = torch.max(item_probs, dim=1)
                item_confidence = item_confidence.item()
                item_idx = item_idx.item()
                item_label = item_model.config.id2label[item_idx]

            if item_confidence >= self.CONFIDENCE_THRESHOLD:
                results.append((category_label, item_label))
            else:
                results.append((category_label, None))

        return results

    def generate_recipe(self, classified_items: list[tuple]) -> str:
        """
        Creates a recipe based on the classified ingredients.

        Parameters:
        classified_items (list[tuple]): List of (category, item) tuples.

        Returns:
        str: Recipe generated by Ollama model.
        """
        ingredients = [item if item else category for category, item in classified_items]
        ingredient_list = ", ".join(ingredients)

        response = ollama.chat(model=self.model_name, messages=[
            {
                'role': "user",
                'content': f"Create a unique recipe using the following ingredients: {ingredient_list}."
            },
        ])
        return response['message']['content']

# Example usage:
# Load images
images = [Image.open("imgs/apple.jpg")]

# Initialize classifier and classify images
classifier = FoodImageClassifier()
classified_items = classifier.classify_images(images)

# Generate and print recipe
recipe = classifier.generate_recipe(classified_items)
print(recipe)

Here's a unique recipe that combines apples with fruit:

**Autumn Spice Apple Crisp with Caramelized Pecans and Cranberry Compote**

Ingredients:

For the crisp topping:

* 1 cup rolled oats
* 1/2 cup brown sugar
* 1/2 cup chopped pecans
* 1 tablespoon cinnamon
* 1/2 teaspoon nutmeg
* 1/4 teaspoon salt

For the apple filling:

* 6-8 medium-sized apples, peeled and sliced (Granny Smith or a mix of sweet and tart varieties)
* 1/4 cup granulated sugar
* 2 tablespoons all-purpose flour
* 1 teaspoon ground cinnamon
* 1/4 teaspoon ground nutmeg

For the caramelized pecans:

* 1/2 cup chopped pecans
* 1 tablespoon unsalted butter
* 1 tablespoon brown sugar

For the cranberry compote:

* 1 cup fresh or frozen cranberries
* 2 tablespoons granulated sugar
* 1 tablespoon orange juice
* 1 tablespoon honey

Instructions:

1. Preheat your oven to 375°F (190°C).
2. In a large bowl, mix together the oat topping ingredients until well combined.
3. In a separate bowl, combine the apple filling ingredien